<a href="https://colab.research.google.com/github/plushvoxel/Project-Lernende-Agenten-colab/blob/master/LSTM_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import random
import numpy as np
import tensorflow as tf
from tarfile import open as taropen
from urllib import request
from struct import unpack
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM

In [0]:
def create_data(trainP = 60, valP= 20, testP = 20):
    pre: trainP + valP + testP <= 100
    DataBaseURL= "https://marble.adhara.uberspace.de/"
    Classes = ["fm", "pager", "smartwares"]
    data = []
    test = 0

    for modulation in Classes:
        filename = "{}.tar".format(modulation)
        request.urlretrieve("https://marble.adhara.uberspace.de/{}_512.tar".format(modulation), filename)
        tar = taropen(filename)
        class_label = Classes.index(modulation)
        for member in tar.getmembers():
            test = test + 1
            with tar.extractfile(member) as f:
                sample = []
                buffer = f.read()
                num_floats = len(buffer)//4
                floats = unpack("f"*num_floats, buffer)
                i = floats[0::2][::4]
                q = floats[1::2][::4]
                for j in range(min(len(i), len(q))):
                    #here happens the scaling or whatever you want
                    sample.append([i[j], q[j]])
                #here happens some cross_feature if you want. just append it to the sample
                
                data.append([np.array(sample), class_label])

    random.shuffle(data)
    #split all data into train, val and test
    size =len(data)
    trainSize = (size * trainP)//100
    valSize = (size * valP)//100
    testSize = (size * testP)//100
    train = data[:trainSize]
    val = data[trainSize: trainSize + valSize]
    test = data[trainSize + valSize: trainSize + valSize + testSize]
    x_train = []
    y_train = []
    x_val = []
    y_val = []
    x_test = []
    y_test = []
    for sample, target in train:
      x_train.append(sample)
      y_train.append(target)
    for sample, target in val:
      x_val.append(sample)
      y_val.append(target)
    for sample, target in test:
      x_test.append(sample)
      y_test.append(target)
    
    
    return np.array(x_train), np.array(x_val), np.array(x_test), y_train, y_val, y_test 

In [0]:
def train_model(model, x_train, y_train, x_val, y_val):
    model.fit(x_train, y_train, epochs = 1, validation_data = (x_val, y_val))

In [0]:
def create_model(x_train, classesAmount = 3):
    opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6)
    model = Sequential()
    shape = x_train.shape[1:]
    print(shape)
    model.add(LSTM(128, input_shape=shape, activation='relu', return_sequences=True))
    model.add(Dropout(0.2))

    model.add(LSTM(128, activation='relu'))
    model.add(Dropout(0.1))

    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.2))

    model.add(Dense(classesAmount, activation='softmax'))

    model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

    return model

In [0]:
x_train, x_val, _x_test, y_train, y_val, y_test = create_data()

In [6]:

model = create_model(x_train)
train_model(model, x_train, y_train, x_val, y_val)

(19704, 128, 2)
(6568, 128, 2)
(6568, 128, 2)
(128, 2)
Train on 19704 samples, validate on 6568 samples
Epoch 1/1
19704/19704 [==============================] - 468s 24ms/step - loss: 6.1120 - acc: 0.4940 - val_loss: 7.3070 - val_acc: 0.5359
